In [ ]:
# import required packages
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform as sf
import random as rd



# Pre processing, dataset description and visualization

In [ ]:
#Loading dataset erasing nan columns 

df = pd.read_csv('Unsupervised Learning 23-24 - Project Dataset.csv', delimiter= ';', header = 0,
                 usecols= lambda col: col not in ['Row', 'Unnamed: 22','Unnamed: 23'], decimal= ',')
# saving number of samples and attributes
[N, M] = df.shape

In [ ]:
#computing correlation in the dataset
corr = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot = False)
#printing the highest correlation values
corr = corr.abs()
corr = corr.unstack()
corr = corr.sort_values(ascending = False)
print(corr[corr < 1].head(10))
# save the image in a file
plt.savefig('correlation.png')



In [ ]:
#correlation of only the last five continuous variables, in order to zoom the plot on the most correlated variables
corr2 = df.iloc[:,range(16,21)].corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr2, annot = True)
plt.savefig('correlation2.png')

In [ ]:
#Boxplot of the dataset (only the numerical variables)
plt.figure(figsize=(10,10))
sns.boxplot(data = df.iloc[:,[0, 16 , 17, 18, 19, 20]])
plt.savefig('boxplot.png')


In [ ]:
# distance matrix with Gower distance
from sklearn.metrics import pairwise_distances
import gower
PM3 = gower.gower_matrix(df)    #gower distance
plt.imshow(PM3) #plotting the distance matrix
plt.colorbar()

In [ ]:
idx = rd.sample(range(7199),30 ) #taking only few samples to have a more intuitive distance matrix
PM4 = gower.gower_matrix(df.iloc[idx,:]) 
plt.imshow(PM4) #plotting the sampled distance matrix
plt.colorbar()
plt.savefig('gower.png')

# Multidimensional scaling
computationally expensive, it will take a while to run

In [ ]:
# Multidimensional scaling with Gower distance to visualize the dataset
from sklearn.manifold import MDS
embedding = MDS(n_components=2, normalized_stress='auto', dissimilarity='precomputed') #setting the MDS model
pairdist2 = pairwise_distances(PM3, metric='precomputed') #pairwise distance with Gower distance (PM3)
df_transformed2 = embedding.fit_transform(pairdist2) #fitting the model
plt.scatter(df_transformed2[:,0], df_transformed2[:,1]) #plotting the MDS

In [ ]:
#Plot of mds of first two components with seaborn scatterplot
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], color='green')
plt.title("MDS with Gower distance")    
plt.xlabel("First component")
plt.ylabel("Second component")
#save the image
plt.savefig('MDS.png')

this will not be used in the following analysis, it's just to show the 3D MDS. Computationally very expensive

In [ ]:
# MDS in 3D (n_components = 3)
embedding_3d = MDS(n_components=3, normalized_stress='auto', dissimilarity='precomputed') #setting the MDS model
pairdist3 = pairwise_distances(PM3, metric='precomputed') #pairwise distance with Gower distance (PM3)
df_transformed3 = embedding_3d.fit_transform(pairdist3) #fitting the model

In [ ]:
#plotting the 3D MDS
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_transformed3[:,0], df_transformed3[:,1], df_transformed3[:,2], marker='.', s=10, c='green')
plt.title("MDS with Gower distance")
plt.xlabel("First component")
plt.ylabel("Second component")
#add third component label
ax.set_zlabel('Third component')
plt.savefig('MDS3D.png')
plt.show()


In [ ]:
# TSNE plot to visualize the dataset
from sklearn.manifold import TSNE
# define the function to plot TSNE
def TSNEPlot(dataset, labels):
    dist_matrix = gower.gower_matrix(dataset) #compute Gower distance matrix
    tsne = TSNE(n_components=2, verbose=0, perplexity=20, n_iter=300, metric="precomputed", init='random') #setting the TSNE model
    tsne_results = tsne.fit_transform(dist_matrix) #fitting the model
    # plot the first two components
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x=tsne_results[:, 0], y=tsne_results[:, 1], color= 'green')
    plt.title("TSNE")
    plt.xlabel("First component")
    plt.ylabel("Second component")
    plt.savefig('TSNE.png')
    plt.show()

In [ ]:
#plotting the TSNE
labels = np.ones(N) #setting all the labels to 1
TSNEPlot(df,labels)

# Anomaly detection

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
#set seed
np.random.seed(42)
isoF =IsolationForest(contamination= 'auto') #setting the model  
isoF.fit(df)    #fitting the model
#saving scores for later 
isof_scores = isoF.decision_function(df)
#print(scores)
#sorted_idx = np.argsort(scores) #sorting the scores
#print(sorted_idx)
#sorted_scores = scores[sorted_idx]
#print(sorted_scores)

# IsoF outliers' prediction
classification = isoF.predict(df)
tot_outliers = sum(classification == -1) #counting the outliers
print("Total outliers: ", tot_outliers)
print("Percentage of outliers: ", tot_outliers/N)



In [ ]:
# visualization of the result with MDS
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue=classification, palette= ['red', 'green'])
plt.legend(["Inlier", "Outlier"])
plt.title("Isolation Forest")
plt.savefig('IsolationForest.png')

## Local Outlier Factor (LOF)

In [ ]:
# LOF algorithm with default settings
np.random.seed(42)  #set seed
from sklearn.neighbors import LocalOutlierFactor
Lof = LocalOutlierFactor(metric='precomputed') #setting the model
Lof = Lof.fit(PM3)  #fitting the model
labels_lof = Lof.fit_predict(PM3)   #predicting the labels

print("Number of outliers: ", sum(labels_lof== -1))
print("Number of inliers: ", sum(labels_lof== 1))
print("Percentage of outliers: ", sum(labels_lof== -1)/N)

In [ ]:
# plot the lof distances
lof_distances, _ = Lof.kneighbors(PM3) #retrieve the distances
plt.plot(lof_distances)
plt.title("Lof distances graph")

In [ ]:
# sort the distances
sort_dist = np.sort(lof_distances,axis=0, )
# plot the sorted distances
plt.plot(sort_dist[:,-1],) #from the plot we clearly see two knees, isolation forest algorithm stops on the second knees ( around 200 outliers) while Lof algorithm stops on the first knee if in 'auto'

#add vertical and horizontal lines to see the two knees
plt.axvline(x=6920, color='k', linestyle='--')
plt.axhline(y=sort_dist[6920,-1], color='k', linestyle='--')
plt.axvline(x=7200-820, color='k', linestyle='--')
plt.axhline(y=sort_dist[7200-820,-1], color='k', linestyle='--')

#add a marker in the intersections
plt.plot(6920, sort_dist[6920,-1], 'o', color='r')
plt.plot(7200-820, sort_dist[7200-820,-1], 'o', color='k')

print("Total outliers: ",N-6920)
print("Percentage of outliers: ", (N-6920)/N)
plt.title("Lof distances graph")
plt.xlabel("Samples")
plt.ylabel("Distances")
plt.savefig('LofDistances.png')


In [ ]:
scores = Lof.negative_outlier_factor_ #get the scores

#plt.plot(scores)
sort_scores = np.sort(scores,axis=0 )
print(sort_scores[-10:])
# verify the distribution of the scores and visualize it with a histogram
sort_descend_scores = np.flip(sort_scores,axis=0)
plt.hist(sort_scores, bins=50, color='blue')
plt.title("Histogram of the scores")
plt.xlim(-9,9)

In [ ]:
# MDS plot of the LOF result
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue= labels_lof, palette= ['black', 'yellow'])
plt.legend([ "Outlier","not Outlier"])
plt.title("Local Outlier Factor")

In [ ]:
# repeating LOF with isolation forest contamination
np.random.seed(42)
Lof2 = LocalOutlierFactor(metric='precomputed', contamination= 0.038)  #setting the model with the new contamination
Lof2 = Lof2.fit(PM3)    #fitting the model
labels_lof2 = Lof2.fit_predict(PM3)  #predicting the labels

print("Number of outliers: ", sum(labels_lof2== -1))
print("Number of inliers: ", sum(labels_lof2== 1))

In [ ]:
# MDS plot of the LOF result with the new contamination
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue= labels_lof2, palette= ['red', 'green'])
plt.legend([ "Inlier","Outlier"])
plt.title("Local Outlier Factor")
plt.savefig('LOF.png')


## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score


In [ ]:
from kneed import KneeLocator
# set seed
np.random.seed(42)
# Finding eps and min_samples that maximize silhouette score
sil = []
knee_y_list = []
for i in range(2,20):
    nbrs = NearestNeighbors(n_neighbors=i+1, metric= 'precomputed').fit(PM3) # fit the NN model        
    distances, indexes = nbrs.kneighbors(PM3)   # retrieve the distances and the indexes   
    sorted_distances = np.sort(distances[:,-1], axis=0) # sort the distances
    x = np.arange(len(sorted_distances))
    knee = KneeLocator(x, sorted_distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')  # find the knee of the sorted distances  
    knee_x = knee.knee
    knee_y = knee.knee_y
    knee_y_list.append(knee_y)
    dbscan = DBSCAN(eps=knee_y, min_samples=i , metric='precomputed').fit(PM3) # apply DBSCAN with the knee distance and the number of neighbors
    dbscan_labels = dbscan.labels_ # retrieve the labels
    # setting all the inliers with label = 1 and all the outliers with label = -1
    for j in range(N):
        if dbscan_labels[j] == -1:
            dbscan_labels[j] = -1
        else: dbscan_labels[j] = 1
    sil.append(silhouette_score(PM3, dbscan_labels, metric='precomputed') )  # compute the silhouette score
    
plt.plot(range(2,20),sil, 'o--', color='r') # plot the silhouette score
#plt.plot(range(2,20),knee_y_list, 'o--', color='b') # plot the knee


In [ ]:
print("Epsilon: " ,knee_y_list[8]) # retrieve the epsilon that maximize the silhouette score
print("min_samples: ", 10) # retrieve the min_samples that maximize the silhouette score
plt.plot(range(2,20),sil, 'o--', color='r') # plot the silhouette score
plt.xlabel("Number of min_samples")
plt.ylabel("Silhouette score")
plt.title("Silhouette score for DBSCAN")
plt.savefig('DBSCAN-SIL.png')


In [ ]:
# plot the NN sorted distances with the chosen parameters
nbrs = NearestNeighbors(n_neighbors=11, metric= 'precomputed').fit(PM3)
distances, indexes = nbrs.kneighbors(PM3) # retrieve the distances and the indexes
sorted_distances = np.sort(distances[:,-1], axis=0) # sort the distances
plt.plot(sorted_distances) # plot the sorted distances
plt.axvline(x=6830, color='k', linestyle='--')
plt.axhline(y=sorted_distances[6830], color='k', linestyle='--')
print(sorted_distances[6830])
plt.title("DBSCAN distances graph")
plt.xlabel("Samples")
plt.ylabel("Distances")
plt.savefig('DBSCAN-knee.png')


In [ ]:
# DBSCAN with the chosen parameters
np.random.seed(42)
nbrs = NearestNeighbors(n_neighbors=11, metric= 'precomputed').fit(PM3) # fit the NN model
dbscan = DBSCAN(eps=0.028174572, min_samples=10 , metric='precomputed').fit(PM3) # apply DBSCAN with eps corresponding to 3.8% of outliers 

# Retrieve the labels
dbscan_labels = dbscan.labels_
#how many clusters
print("Number of clusters: ", len(set(dbscan_labels)))
#setting all the inliers with label = 1 and all the outliers with label = -1
for i in range(N):
  #print(i)
  if dbscan_labels[i] == -1:
    dbscan_labels[i] = -1
  else: dbscan_labels[i] = 1
#print(dbscan_labels)
print("Number of outliers: ", sum(dbscan_labels== -1))
print("Number of inliers: ", sum(dbscan_labels== 1))
print("Percentage of outliers: ", sum(dbscan_labels== -1)/N)
print("Silhouette: ", silhouette_score(PM3, dbscan_labels, metric='precomputed') )

In [ ]:
# MDS plot of the DBSCAN result
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue= dbscan_labels, palette= ['red', 'green'])
plt.legend([ "not Outlier","Outlier"])
plt.title("DBSCAN")
plt.savefig('DBSCAN.png')

## Reconstruction method: PCA

In [ ]:
from sklearn.decomposition import PCA

np.random.seed(42)

NCOMP = 15  # number of components

# Apply PCA
pca = PCA(n_components=NCOMP)
pca_result = pca.fit_transform(PM3) #fitting the model on the distance matrix
print('PCA: explained variation per principal component: {}'.format(pca.explained_variance_ratio_.round(3)))
print('Total explained variation: ', pca.explained_variance_ratio_.sum())
PM_reconstructed = pca.inverse_transform(pca_result) #reconstructing the distance matrix
RE_mean = np.mean(np.square(PM_reconstructed-PM3)) #computing the mean squared reconstruction error
RE = np.abs(PM_reconstructed-PM3) #computing the absolute reconstruction error
print(RE.shape)
#print(RE)
print('Reconstructed error: %.8f' % RE_mean)
RE = np.mean(RE, axis=1)  #computing the mean reconstruction error for each sample
plt.plot(RE)

In [ ]:
RE_sorted = np.sort(RE) #sorting the reconstruction errors
plt.plot(RE_sorted) #plotting the sorted reconstruction errors

In [ ]:
# Chi-squared method for detecting outliers
from scipy.stats import chi2

alpha = 0.05 # significance value
sq_proj = []
for i in range(NCOMP):
  sq_proj.append((pca_result[:,i]**2)/(np.sqrt(pca.explained_variance_[i]))) #computing the squared projection normalized by the eigenvalues
  
print(sq_proj[0].shape) #checking the shape of the first element

summed_proj = np.sum(sq_proj, axis=0) #summing the squared projections
print(summed_proj.shape)
tresh = chi2.ppf(1- alpha, NCOMP) #computing the threshold
print(tresh)


CHI_labels = np.ones(N)
CHI_labels[summed_proj > tresh] = -1 # if the sum of the squared projections is greater than the threshold, the sample is an outlier
print("number of outliers: ", sum(CHI_labels == -1))
print("number of inliers: ", sum(CHI_labels == 1))
print("percentage of outliers: ", sum(CHI_labels == -1)/N)

In [ ]:
# MDS plot of the PCA result
sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue= CHI_labels, palette= ['red', 'green'])
plt.legend([ "not Outlier","Outlier"])
plt.title("PCA")
plt.savefig('PCA3.png')

Let's compute the rand index of the various methods

In [ ]:
from sklearn.metrics.cluster import rand_score

# retrieve the labels of the methods
ISOF_labels = classification
LOF_labels = labels_lof2
DBSCAN_labels = dbscan_labels
CHI_labels

# compute the rand index between the methods

rand_index1 = rand_score(ISOF_labels, LOF_labels) #rand index between ISOF and LOF
print("Rand score between ISOF and LOF ",rand_index1.round(3))

rand_index2 = rand_score(LOF_labels, DBSCAN_labels) #rand index between LOF and DBSCAN
print("Rand score between DBSCAN and LOF ",rand_index2.round(3))

rand_index3 = rand_score(LOF_labels, CHI_labels)  #rand index between LOF and PCA
print("Rand score between PCA and LOF ",rand_index3.round(3))

rand_index4 = rand_score(ISOF_labels, DBSCAN_labels)   #rand index between ISOF and DBSCAN
print("Rand score between ISOF and DBSCAN ",rand_index4.round(3))

rand_index5 = rand_score(ISOF_labels, CHI_labels)   #rand index between ISOF and PCA
print("Rand score between ISOF and PCA ",rand_index5.round(3))

rand_index6 = rand_score(CHI_labels, DBSCAN_labels)   #rand index between PCA and DBSCAN
print("Rand score between PCA and DBSCAN ",rand_index6.round(3))

In [ ]:
#Find out which observation are not outlier according to all algorithms
total_labels = ISOF_labels + LOF_labels + DBSCAN_labels + CHI_labels #summing the labels 
#visualize on the MDS plot
PAL = ['red', 'blue', 'black', 'orange', 'green']
print("2 algorithms flagged as outliers, 2 as inliers" ,sum(total_labels == 0))
print("3 algorithms flagged as outliers, 1 as inliers" ,sum(total_labels == -2))
print("1 algorithms flagged as outliers, 3 as inliers" ,sum(total_labels == 2))
print("4 algorithms flagged as outliers" ,sum(total_labels == -4))
print("4 algorithms flagged as inliers" ,sum(total_labels == 4))

In [ ]:
#plotting the classification according to all algorithms
fig = sns.scatterplot(x = df_transformed2[:,0], y = df_transformed2[:,1], hue= total_labels, palette= PAL)

plt.legend(["Outlier", "3 Outlier 1 Inlier","2 Outlier 2 Inlier","1 Outlier 3 Inlier" ,"Inlier"])
plt.title("Classification according to all algorithms")
plt.savefig('Classification.png')

the legend colors are wrong because of seaborn scatterplot, they were corrected manually by us

In [ ]:
# Selecting ISOF for the outliers scores
plt.plot(isof_scores) #plotting the isof scores
#sorted isof scores
sorted_isof = np.sort(isof_scores)
print(" min:   ",min(isof_scores))
print(" max:   ",max(isof_scores))
plt.plot(sorted_isof)

In [ ]:
#Saling the scores from 0 to 100
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 100)) #scaling the scores from 0 to 100 using MinMaxScaler
isof_scores = isof_scores.reshape(-1,1)
isof_scores = scaler.fit_transform(isof_scores) #fitting the scaler
isof_scores = 100 - isof_scores #reversing the scores
plt.plot(isof_scores)
#append to the dataset as a new column
df['isof_scores'] = isof_scores



In [ ]:
df.head()
#save the new dataset
df.to_csv('new_dataset.csv', index=True)
